In [3]:
import pandas as pd
from matplotlib import pyplot as plt
import torch
from torch import nn

import dataloaders
from NBEATS import *

[*********************100%***********************]  1 of 1 completed

Ticker          TSLA
Date                
2012-02-10  2.073333
2012-02-13  2.099333
2012-02-14  2.211333
2012-02-15  2.240000
2012-02-16  2.278667


ValueError: could not broadcast input array from shape (50,1) into shape (50,)

In [ ]:
data_filepath = r"C:\Users\Atakan\atakan_python\WQU_DL\ALGOTRADE\static_files\btc_1min.csv"
df = pd.read_csv(data_filepath, header=0, index_col='Timestamp', parse_dates=['Timestamp'])
df.index = pd.to_datetime(df.index, unit='s')

start_date = "2019-01-01"
end_date = "2023-03-31"

df = df.loc[start_date:end_date, :]
print(df.head())

C:\Users\Atakan\AppData\Local\Temp\ipykernel_16800\739752483.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(data_filepath, header=0, index_col='Timestamp', parse_dates=['Timestamp'])


                        Open     High      Low    Close  Volume_(BTC)  \
Timestamp                                                               
2019-01-01 00:00:00  3693.85  3698.79  3693.85  3698.00      5.491289   
2019-01-01 00:01:00  3694.72  3694.72  3690.65  3690.65      9.500151   
2019-01-01 00:02:00  3689.73  3689.73  3686.62  3686.62      0.965966   
2019-01-01 00:03:00  3692.85  3692.85  3688.32  3692.35      0.296662   
2019-01-01 00:04:00  3692.35  3692.35  3690.34  3690.34      0.111622   

                     Volume_(Currency)  Weighted_Price  
Timestamp                                               
2019-01-01 00:00:00       20301.156505     3696.974936  
2019-01-01 00:01:00       35080.265871     3692.600865  
2019-01-01 00:02:00        3562.371230     3687.884698  
2019-01-01 00:03:00        1095.220713     3691.813285  
2019-01-01 00:04:00         412.065433     3691.614849  


In [ ]:
series = df.loc[:, 'Close']
series.head()

# Get rid of NaN values
num_nans = series.isna().sum()
print(num_nans)

series.fillna(method='ffill', inplace=True)
num_nans = series.isna().sum()
print(num_nans)

25541
0


C:\Users\Atakan\AppData\Local\Temp\ipykernel_16800\2367134223.py:8: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  series.fillna(method='ffill', inplace=True)


In [ ]:
# --- Custom Loss Functions ---
def sMAPE_loss(forecast, target, eps=1e-6):
    # sMAPE = 200/H * mean(|F - y| / (|F| + |y|))
    numerator = torch.abs(forecast - target)
    denominator = torch.abs(forecast) + torch.abs(target) + eps
    return 200 * torch.mean(numerator / denominator)

In [ ]:
# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

M = 3 # Number of stacks in the network
K = 30  # Number of blocks in the stack
H = 10  # Forecast horizon
n = 5  # Look back n times the forecast horizon

# Create the dataloaders
train_loader, val_loader, test_loader = dataloaders.NBEATS_Data_Loader(series, H, n, val_size=0.2, test_size=0.1, batch_size=1024)

num_epochs = 30

# Initialize model
model = NBEATS(M, K, H, n).to(device)

# Choose loss function and optimizer
loss_func = sMAPE_loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Train the model
train_NBEATS(train_loader, val_loader, model, loss_func, optimizer, device, num_epochs)

Using device: cuda


RuntimeError: The size of tensor a (1024) must match the size of tensor b (10) at non-singleton dimension 1